In [16]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
import boto3
import os
from dotenv import load_dotenv
import pandas as pd

In [17]:
load_dotenv()
s3r = boto3.client('s3', 
                   aws_access_key_id=os.environ["DEV_ACCESS_KEY"], 
                   aws_secret_access_key=os.environ["DEV_SECRET_ACCESS_KEY"])

### Download test data from dev s3

In [7]:
name = f"df_test_v1.xlsx"
fullname = f'swapnil/form_selection_core/excel/' + name
s3r.download_file(Bucket = "datainsights-shared-coupadev-com", Key = fullname, Filename = name)

## load test data

In [19]:
test_data_path = "df_test_v1.xlsx"
test_data = pd.read_excel(test_data_path)
test_data.head(2)

,instance,easy_form_widget_response_id,updated_at,easy_form_id,easy_form_widget_id,easy_form_response_id,user_submitted_description,easy_form_widget_response_type,backing_attribute,field_name,...,subject_type,status,easy_form_name,easy_form_model,easy_form_status,easy_form_description,form_type,requistion_line_id,header_id,channel
0,cbre,NaN,2023-11-01 11:09:43,NaN,NaN,NaN,"""Warning Trip Hazard"" Sign 210mm x 148mm",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,freeform,5889876.0,3013667.0,freeform
1,monash,NaN,2023-09-11 00:20:35,NaN,NaN,NaN,DEBDEN DAYPLANNER refil,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,freeform,1091278.0,479534.0,freeform


In [ ]:
all_personas = {
    'verbose':
        'You write longer complete sentences that are friendlier and generally include punctuation.',
    'concise':
        'You write direct queries with minimal non-essential text. You usually omit capitalization and filler phrases',
    'step-by-step':
        'You summarize the goal of each step before explaining the detailed instructions.',
    'casual':
        'You use informal language that may not directly reference all the details',
    'formal':
        'You like to provide detailed information and formal language.',
    'spelling-mistake':
        'You often make spelling mistakes'
}

def persona_dict(persona):
    assert persona in all_personas
    return {"persona_type": persona, "persona_description": all_personas[persona]}

# example
print(persona_dict('concise'))

In [ ]:
prompt = f"""\
You work for a corporate business and as an employee, you want to buy/procure some goods or services on behalf of your organisation.
You will be given a description of item/service from which you need to construct your request. You will put the request into an AI Assitant.
Extract the basic information such as amount, quantity, business justification etc if present, from the given fields in order to create a simple one line request using them.
Use only minimal information as you can provide the details later.

Make the request concise and simple. Use given description judiciously and include important information. Feel free to add more information.
If any additional preferences are given, use them during sentence construction as well.
Instructions:
- You are a person having a persona that matches the given persona description. Act accordingly.
item_or_service_description:
{description}
Persona description:
{persona}:{persona_description}
Request:"""


In [ ]:
persona = "spelling-mistake"
persona_description= all_personas[persona]
description = 'laptop'
print(prompt)

In [ ]:
llm.invoke([prompt])

In [ ]:
llm_reflection = initialize_llm(temperature=0, streaming=True)


In [ ]:
prompt_reflection = f"""'I need to purchase a laptop for work purposes.' 
do you think above request is made by a person having characteristics :{persona_description}. give the answer as a tuple - (yes/no, reason)
"""

print(prompt_reflection)

In [ ]:
llm.invoke([prompt_reflection])